### Домашнее задание по теме «Рекомендации на основе содержания»

1. Использовать dataset MovieLens
2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:  
-TF-IDF на тегах и жанрах  
-Средние оценки (+ median, variance, etc.) пользователя и фильма
3. Оценить RMSE на тестовой выборке

In [178]:
import pandas as pd
import numpy as np


from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline
from tqdm import tqdm_notebook

In [179]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [180]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId')

In [181]:
# полностью объединенные данные фильмы, рейтинги, тэги
data = movies_with_ratings.join(tags.set_index('movieId'), on='movieId', lsuffix='_left', rsuffix='_right')

In [182]:
# удаляю признаки которые не будут использоваться 
data = data.drop(['timestamp_left', 'userId_right', 'timestamp_right'], axis=1)

- сколько жанров
- сколько тегов
- количество оценок у фильма
- средняя оценка фильма
- средняя оценка пользователя
- tfidf жанры
- tfidf теги
- выделить год выпуска фильма

In [183]:
# считаю сколько у фильма жанров
num_genres = []

for g in data.genres:
    gg = g.split('|')
    num_genres.append(len(gg))
    
# создаю новый признак с количеством жанров
data['num_genres'] = num_genres

In [184]:
# в фильмах у которых нет тегов присаою значение 0
data.loc[data.tag.isna(), 'tag'] = 0

In [185]:
# достанем по каждому фильму количество тегов
title_num_tags = {}

for title, group in data.groupby('title'):
    title_num_tags[title] = group.tag.unique().shape[0]

In [186]:
# функция которая передаст значения из словаря в общий датафрейм
def new(row):
    for i, j in title_num_tags.items():
        if row == i:
            return j   

In [187]:
# создаем новый признак с количеством тегов и применяем функцию переноса данных
data['num_tags'] = data.title.apply(new)

In [188]:
# достанем по каждому фильму количество оценок
title_num_ratings = {}

for title, group in data.groupby('title'):
    title_num_ratings[title] = group.groupby('userId_left').userId_left.unique().shape[0]

In [189]:
# функция которая передаст значения из словаря в общий датафрейм
def new_rat(row):
    for i, j in title_num_ratings.items():
        if row == i:
            return j 

In [190]:
# создаем новый признак с количеством рейтингов и применяем функцию переноса данных
data['num_ratings'] = data.title.apply(new_rat)

In [191]:
# считаем средний рейтинг на каждый фильм
title_mean_rating = {}

for title, group in data.groupby('title'):
    title_mean_rating[title] = group.rating.mean()

In [192]:
# функция которая передаст значения из словаря в общий датафрейм
def rat_mean(row):
    for i, j in title_mean_rating.items():
        if row == i:
            return j 

In [193]:
data['mean_rating'] =  data.title.apply(rat_mean)

In [194]:
# считаем среднюю оценку пользователя
mean_rating_user = {}

for title, group in data.groupby('userId_left'):
    mean_rating_user[title] = group.rating.mean()

In [195]:
# функция которая передаст значения из словаря в общий датафрейм
def user_mean(row):
    for i, j in mean_rating_user.items():
        if row == i:
            return j 

In [196]:
data['mean_rat_user'] =  data.userId_left.apply(user_mean)

In [56]:
import re

In [123]:
def take_year(row):
    result = re.findall(r'(\d{4})', row)
    return result[0]

построил предсказание оценки только на двух признаках: пользователь и фильм. Без обработки данных и создания новых признаков

In [197]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [41]:
X = data[['userId_left', 'movieId']]
y = data['rating']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [43]:
forest_model = RandomForestRegressor()
forest_model.fit(X, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [60]:
y_pred = forest_model.predict(X_test)

In [61]:
print(f'RMSE:{mean_squared_error(y_test, y_pred, squared=False):.2f}')

0.20422147416168707

построил предсказание оценки после создания новых признаков

In [198]:
data = data[data.userId_left.notna()]

In [173]:
X = data[['userId_left', 'movieId', 'num_genres', 'num_tags', 'num_ratings', 'mean_rating', 'mean_rat_user']]
y = data['rating']

In [174]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [175]:
forest_model = RandomForestRegressor()
forest_model.fit(X, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [176]:
y_pred = forest_model.predict(X_test)

In [177]:
print(f'RMSE:{mean_squared_error(y_test, y_pred, squared=False):.2f}')

RMSE:0.16


т.о. создание новых признаков позволило улучшить предсказание, о чем говорит показатель меньшей ошибки

Теперь в данные добавляю обработанные данные по tfidf признак жанры, и построю предсказание на общих данных

In [200]:
# задаю функцию в которую будет подаваться значение, и она будет создавать сроку с разделенными словами
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [201]:
# создаю список значений жанров разделенных на слова пробелом
movie_genres = [change_string(g) for g in data.genres.values]

In [202]:
# считаю количество вхождений слов 
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

In [203]:
# перевожу векторы жанров в взвешенное значение
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [206]:
X_train_tfidf.toarray().shape

(285762, 20)

In [209]:
data1 = pd.DataFrame(X_train_tfidf.toarray())

In [219]:
data1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.348743,0.548508,0.527195,0.255409,0.0,0.0,0.0,0.484096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.348743,0.548508,0.527195,0.255409,0.0,0.0,0.0,0.484096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.348743,0.548508,0.527195,0.255409,0.0,0.0,0.0,0.484096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.348743,0.548508,0.527195,0.255409,0.0,0.0,0.0,0.484096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.348743,0.548508,0.527195,0.255409,0.0,0.0,0.0,0.484096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [220]:
# добавляю tfidf в общий датафрейм
data2 = data.join(data1)

In [229]:
# определяю Х у
X_gen = data2[[      'movieId',   'userId_left',  'num_genres',      'num_tags',
         'num_ratings',   'mean_rating', 'mean_rat_user',               0,
                     1,               2,               3,               4,
                     5,               6,               7,               8,
                     9,              10,              11,              12,
                    13,              14,              15,              16,
                    17,              18,              19]]
y_gen = data2.rating

In [230]:
X_train, X_test, y_train, y_test = train_test_split(X_gen, y_gen, test_size=0.2)

forest_model = RandomForestRegressor()
forest_model.fit(X_train, y_train)

y_pred = forest_model.predict(X_test)

In [231]:
print(f'RMSE:{mean_squared_error(y_test, y_pred, squared=False):.2f}')

RMSE:0.44


Выводы: по результатам эксперементов, получилось что модель с наименьшей ошибкой RMSE:0.16, построена на данных которые были получены созданием новых признаков. Добавление tfidf по жанрам в общий датафрейм, ухудшает предсказание.